# Linear Regression using Pyspark

In [1]:
#create sparksession object
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('lin_reg').getOrCreate()

In [2]:
#import Linear Regression from spark's MLlib
from pyspark.ml.regression import LinearRegression

In [3]:
#Load the dataset
df=spark.read.csv('Linear_regression_dataset.csv',inferSchema=True,header=True)

In [4]:
#validate the size of data
print((df.count(), len(df.columns)))

(1232, 6)


In [5]:
#explore the data
df.printSchema()

root
 |-- var_1: integer (nullable = true)
 |-- var_2: integer (nullable = true)
 |-- var_3: integer (nullable = true)
 |-- var_4: double (nullable = true)
 |-- var_5: double (nullable = true)
 |-- output: double (nullable = true)



In [7]:
#view statistical measures of data 
df.describe().show(5,False)

+-------+-----------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|var_1            |var_2            |var_3             |var_4               |var_5               |output             |
+-------+-----------------+-----------------+------------------+--------------------+--------------------+-------------------+
|count  |1232             |1232             |1232              |1232                |1232                |1232               |
|mean   |715.0819805194806|715.0819805194806|80.90422077922078 |0.3263311688311693  |0.25927272727272715 |0.39734172077922014|
|stddev |91.5342940441652 |93.07993263118064|11.458139049993724|0.015012772334166148|0.012907228928000298|0.03326689862173776|
|min    |463              |472              |40                |0.277               |0.214               |0.301              |
|max    |1009             |1103             |116               |0.373               |0.294               |0.491

In [8]:
#sneak into the dataset
df.head(3)

[Row(var_1=734, var_2=688, var_3=81, var_4=0.328, var_5=0.259, output=0.418),
 Row(var_1=700, var_2=600, var_3=94, var_4=0.32, var_5=0.247, output=0.389),
 Row(var_1=712, var_2=705, var_3=93, var_4=0.311, var_5=0.247, output=0.417)]

In [9]:
#import corr function from pyspark functions
from pyspark.sql.functions import corr

In [10]:
# check for correlation
df.select(corr('var_1','output')).show()

+-------------------+
|corr(var_1, output)|
+-------------------+
| 0.9187399607627283|
+-------------------+



In [11]:
#import vectorassembler to create dense vectors
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

In [12]:
#select the columns to create input vector
df.columns

['var_1', 'var_2', 'var_3', 'var_4', 'var_5', 'output']

In [13]:
#create the vector assembler 
vec_assmebler=VectorAssembler(inputCols=['var_1', 'var_2', 'var_3', 'var_4', 'var_5'],outputCol='features')

In [14]:
#transform the values
features_df=vec_assmebler.transform(df)

In [15]:
#validate the presence of dense vectors 
features_df.printSchema()

root
 |-- var_1: integer (nullable = true)
 |-- var_2: integer (nullable = true)
 |-- var_3: integer (nullable = true)
 |-- var_4: double (nullable = true)
 |-- var_5: double (nullable = true)
 |-- output: double (nullable = true)
 |-- features: vector (nullable = true)



In [16]:
#view the details of dense vector
features_df.select('features').show(5,False)

+------------------------------+
|features                      |
+------------------------------+
|[734.0,688.0,81.0,0.328,0.259]|
|[700.0,600.0,94.0,0.32,0.247] |
|[712.0,705.0,93.0,0.311,0.247]|
|[734.0,806.0,69.0,0.315,0.26] |
|[613.0,759.0,61.0,0.302,0.24] |
+------------------------------+
only showing top 5 rows



In [17]:
#create data containing input features and output column
model_df=features_df.select('features','output')

In [18]:
model_df.show(5,False)

+------------------------------+------+
|features                      |output|
+------------------------------+------+
|[734.0,688.0,81.0,0.328,0.259]|0.418 |
|[700.0,600.0,94.0,0.32,0.247] |0.389 |
|[712.0,705.0,93.0,0.311,0.247]|0.417 |
|[734.0,806.0,69.0,0.315,0.26] |0.415 |
|[613.0,759.0,61.0,0.302,0.24] |0.378 |
+------------------------------+------+
only showing top 5 rows



In [18]:
#size of model df
print((model_df.count(), len(model_df.columns)))

(1232, 2)


### Split Data - Train & Test sets


In [19]:
#split the data into 70/30 ratio for train test purpose
train_df,test_df=model_df.randomSplit([0.7,0.3])

In [20]:
print((train_df.count(), len(train_df.columns)))

(843, 2)


In [21]:
print((test_df.count(), len(test_df.columns)))

(389, 2)


In [22]:
train_df.describe().show()

+-------+-------------------+
|summary|             output|
+-------+-------------------+
|  count|                843|
|   mean| 0.3981637010676153|
| stddev|0.03294758324054597|
|    min|              0.311|
|    max|              0.491|
+-------+-------------------+



## Build Linear Regression Model 

In [23]:
#Build Linear Regression model 
lin_Reg=LinearRegression(labelCol='output')

In [24]:
#fit the linear regression model on training data set 
lr_model=lin_Reg.fit(train_df)

In [25]:
lr_model.intercept

0.19182762797333336

In [26]:
print(lr_model.coefficients)

[0.000340839517716,4.79714814651e-05,0.000188236319406,-0.683084289192,0.520925094105]


In [27]:
training_predictions=lr_model.evaluate(train_df)

In [28]:
training_predictions.meanSquaredError

0.0001479324335891862

In [29]:
training_predictions.r2

0.863563127042712

In [30]:
#make predictions on test data 
test_results=lr_model.evaluate(test_df)

In [31]:
#view the residual errors based on predictions 
test_results.residuals.show(10)

+--------------------+
|           residuals|
+--------------------+
|-0.01339317627364...|
|0.005852942292043195|
|-0.01270570081380...|
|-0.00719060666608...|
|-0.00117340799738...|
|0.003662428381860...|
|-0.01232056280684...|
|-0.00201734580289...|
|0.011105011921430763|
|-0.00108724755114...|
+--------------------+
only showing top 10 rows



In [32]:
#coefficient of determination value for model
test_results.r2

0.8796832901372513

In [33]:
test_results.rootMeanSquaredError

0.011751649289716636

In [34]:
test_results.meanSquaredError

0.0001381012610284975